In [1]:
#Import packages
import earthaccess
import xarray as xr

In [2]:
#Authentication with Earthdata Login
auth = earthaccess.login(strategy="netrc")

In [3]:
#Access land ice height from ATLAS/ICESat-2 V005 (10.5067/ATLAS/ATL06.005), searching for data over western Greenland coast over two weeks in July 2022. The data are provided as HDF5 granules (files) that span about 1/14th of an orbit.

results = earthaccess.search_data(short_name="ATL06",
                                  version="006",
                                  cloud_hosted=True,
                                  temporal = ("2022-07-17","2022-07-31"),
                                  bounding_box = (-51.96423,68.10554,-48.71969,70.70529))

In [4]:
#Use xarray to load the data as a multifile dataset for a single group in the HDF5 file, in this case land ice segments:
ds = xr.open_mfdataset(earthaccess.open(results), group='/gt1l/land_ice_segments')
ds

QUEUEING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/5 [00:00<?, ?it/s]

<xarray.Dataset> Size: 11MB
Dimensions:                (delta_time: 249580)
Coordinates:
  * delta_time             (delta_time) datetime64[ns] 2MB 2022-07-18T01:00:4...
    latitude               (delta_time) float64 2MB dask.array<chunksize=(10000,), meta=np.ndarray>
    longitude              (delta_time) float64 2MB dask.array<chunksize=(10000,), meta=np.ndarray>
Data variables:
    atl06_quality_summary  (delta_time) int8 250kB dask.array<chunksize=(10000,), meta=np.ndarray>
    h_li                   (delta_time) float32 998kB dask.array<chunksize=(10000,), meta=np.ndarray>
    h_li_sigma             (delta_time) float32 998kB dask.array<chunksize=(10000,), meta=np.ndarray>
    segment_id             (delta_time) float64 2MB dask.array<chunksize=(10000,), meta=np.ndarray>
    sigma_geo_h            (delta_time) float32 998kB dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    Description:  The land_ice_height group contains the primary set of deriv...
    data_rate:    Data within this group are sparse.  Data values are provide...

In [8]:
import requests
import xarray as xr
from io import BytesIO
import earthaccess


earthaccess.login(persist=True)


granule_urls = [
    'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc',
    
]


fs = earthaccess.get_requests_https_session()


datasets = []
for url in granule_urls:
    response = fs.get(url, stream=True)
    file_obj = BytesIO(response.content)  # Stream the data into memory
    ds = xr.open_dataset(file_obj, engine='h5netcdf')        # Load into xarray Dataset
    datasets.append(ds)


#combined_ds = xr.concat(datasets, dim="time")  # Combine datasets along the time dimension


ValueError: b'<html>\n ' is not the signature of a valid netCDF4 file